In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import glob
import os
import numpy as np

In [2]:
#from torch.utils.tensorboard import SummaryWriter
from tensorboardX import SummaryWriter
writer=SummaryWriter('/home/are/data_cervical/pytorch_running/third/resnet50/Adam/run3')


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=False):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input,dim=1)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

In [2]:
transform = transforms.Compose(
    [transforms.Resize((224,224)),transforms.ToTensor(),
     transforms.Normalize((0.5941, 0.3818, 0.3370), (0.1508 ,0.1404, 0.1222))])
train_dataset=torchvision.datasets.ImageFolder('/home/are/data_cervical/pytorch_running/3_data/caijian2/train/',transform)
valid_dataset=torchvision.datasets.ImageFolder('/home/are/data_cervical/pytorch_running/3_data/caijian2/valid/',transform)
train_dataset.class_to_idx

{'ONE': 0, 'THREE': 1, 'TWO': 2}

In [3]:
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=32,shuffle=True,num_workers=5)
valid_loader=torch.utils.data.DataLoader(valid_dataset,batch_size=32,shuffle=True,num_workers=5)

In [2]:
model_ft=models.resnet50(pretrained=True)
#print(model_ft)
num_ftrs=model_ft.fc.in_features
model_ft.fc=nn.Linear(num_ftrs,3)  
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft=model_ft.to(device)

print(model_ft)
print(device)
#现在不叫迁移学习，迁移学习要固定前面的参数
#print(model_ft.layer4.parameters())

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [11]:
for m in model_ft._modules:
    print(m)
isinstance(model_ft._modules['layer1'],nn.Sequential)
for m in model_ft._modules['layer1'].children():
    print(m)
    for n in m.children():
        print(n)
        break
    break

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc
Bottleneck(
  (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (downsample): Sequential(
    (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)
Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [6]:
learning_rate=0.001
#weight_class=torch.FloatTensor([10,10,1])
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model_ft.parameters(),weight_decay=1e-4,lr=0.001,betas=(0.9,0.99))

In [9]:
save_path='/home/are/data_cervical/pytorch_running/third/resnet50/Adam'
import time
dataloader={'train':train_loader,'valid':valid_loader}
dataset={'train':train_dataset,'valid':valid_dataset}
best_model_wts=model_ft.state_dict()
best_acc=0.0
num_epochs=200
train_loss=[]
train_acc=[]
valid_loss=[]
valid_acc=[]
times=[]
for epoch in range(num_epochs):
    since=time.time()
    print('epoch:',epoch)
    print('-'*10)
        
    for phase in ['train','valid']:
            
        if phase=='train':
            model_ft.train(True)
        else:
            model_ft.train(False)
        running_loss=0.0
        running_corrects=0
        
        for i, data in enumerate(dataloader[phase], 0):
            print(i)
            inputs, labels = data
            inputs=torch.autograd.Variable(inputs.to(device))  #数据和标签要放到GPU上
            labels=torch.autograd.Variable(labels.to(device))
                
            optimizer.zero_grad()
                # forward + backward + optimize
            outputs = model_ft(inputs)
            loss = criterion(outputs, labels)
            _,preds=torch.max(outputs.data,1)
            if phase=='train':
                loss.backward()
                optimizer.step()
            running_loss+=loss.item()
            running_corrects+=(torch.sum(preds==labels.data)).item()
        print(running_loss)
        print(running_corrects)
        break
           
        print(len(dataset[phase]))
        epoch_loss=running_loss/len(dataset[phase])
            #epoch_loss=running_loss/4463
        epoch_acc=running_corrects/len(dataset[phase])
        if phase=='train':
            train_loss.append(epoch_loss)
            train_acc.append(epoch_acc)
            #writer.add_scalars('loss_train_val',{'train':epoch_loss},epoch)
            #writer.add_scalars('acc_train_val',{'train':epoch_acc},epoch)
            #if epoch%40==0: #每隔一段时间保存一次
             #   state={'net':model_ft.state_dict(),'optimizer':optimizer.state_dict(),'epoch':epoch}
              #  dir=os.path.join(save_path,'epoch_{}.pth'.format(epoch))
               # torch.save(state,dir)
        else:
            valid_loss.append(epoch_loss)
            valid_acc.append(epoch_acc)
            #writer.add_scalars('loss_train_val',{'valid':epoch_loss},epoch)
            #writer.add_scalars('acc_train_val',{'valid':epoch_acc},epoch)
        print('{} loss: {:.8f} acc:{:.8f}'.format(phase,epoch_loss,epoch_acc))
        if phase=='valid' and epoch_acc>best_acc:
            best_acc=epoch_acc
            best_model_wts=model_ft.state_dict()
            
    time_elapsed= time.time()-since
    times.append(time_elapsed)
    print('training time:',time_elapsed)
model_ft.load_state_dict(best_model_wts)
torch.save(best_model_wts,os.path.join(save_path,'resnet50_weight.pth'))
torch.save(model_ft,os.path.join(save_path,'resnet50.pth'))
print('best acc',best_acc)


import matplotlib.pyplot as plt
%pylab inline
x1=range(0,num_epochs)
x2=range(0,num_epochs)
y1=train_loss
y2=train_acc
y3=valid_acc
y4=valid_loss
plt.title('loss VS epoch')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.plot(x1,y1,'b',label='train_loss')
plt.plot(x1,y4,'y',label='valid_loss')
plt.legend()#显示label
#plt.savefig(os.path.join(save_path,'1.jpg'))
plt.show()  #一定要在show之前保存

plt.title('acc VS epoch')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.plot(x2,y2,'c',label='train_acc')
plt.plot(x2,y3,'r',label='valid_acc')
plt.legend()#显示label
#plt.savefig(os.path.join(save_path,'2.jpg'))
plt.show()

with open(os.path.join(save_path,'train_loss.txt'),'w') as f:
    for line in train_loss:
        f.write(str(line)+'\n')
with open(os.path.join(save_path,'valid_loss.txt'),'w') as f:
    for line in valid_loss:
        f.write(str(line)+'\n')
with open(os.path.join(save_path,'train_acc.txt'),'w') as f:
    for line in train_acc:
        f.write(str(line)+'\n')

with open(os.path.join(save_path,'valid_acc.txt'),'w') as f:
    for line in valid_acc:
        f.write(str(line)+'\n')
with open(os.path.join(save_path,'time.txt'),'w') as f:
    for line in times:
        f.write(str(line)+'\n')
with open(os.path.join(save_path,'best.txt'),'w') as f:
    f.write(str(best_acc))

epoch: 0
----------
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
27

KeyboardInterrupt: 

In [8]:

#save_path='/home/are/data_cervical/pytorch_running/third/resnet50/Adam'


In [2]:
import matplotlib.pyplot as plt
%pylab inline
x1=range(0,num_epochs)
x2=range(0,num_epochs)
y1=train_loss
y2=train_acc
y3=valid_acc
y4=valid_loss
plt.title('loss VS epoch')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.plot(x1,y1,'b',label='train_loss')
plt.plot(x1,y4,'y',label='valid_loss')
plt.legend()#显示label
#plt.savefig(os.path.join(save_path,'1.jpg'))
plt.show()  #一定要在show之前保存

plt.title('acc VS epoch')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.plot(x2,y2,'c',label='train_acc')
plt.plot(x2,y3,'r',label='valid_acc')
plt.legend()#显示label
#plt.savefig(os.path.join(save_path,'2.jpg'))
plt.show()



Populating the interactive namespace from numpy and matplotlib


NameError: name 'num_epochs' is not defined

In [25]:
with open(os.path.join(save_path,'train_loss.txt'),'w') as f:
    for line in train_loss:
        f.write(str(line)+'\n')
with open(os.path.join(save_path,'valid_loss.txt'),'w') as f:
    for line in valid_loss:
        f.write(str(line)+'\n')
with open(os.path.join(save_path,'train_acc.txt'),'w') as f:
    for line in train_acc:
        f.write(str(line)+'\n')

with open(os.path.join(save_path,'valid_acc.txt'),'w') as f:
    for line in valid_acc:
        f.write(str(line)+'\n')
with open(os.path.join(save_path,'time.txt'),'w') as f:
    for line in times:
        f.write(str(line)+'\n')
with open(os.path.join(save_path,'best.txt'),'w') as f:
    f.write(str(best_acc))


In [9]:
load_path='/home/are/data_cervical/pytorch_running/third/resnet50/Adam/resnet50_weight.pth'

model_ft.load_state_dict(torch.load(load_path))
dataloader={'train':train_loader,'valid':valid_loader}
dataset={'train':train_dataset,'valid':valid_dataset}
running_corrects=0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
c00=0
c01=0
c02=0
c10=0
c11=0
c12=0
c20=0
c21=0
c22=0
for i, data in enumerate(dataloader['valid'], 0):
    model_ft.eval()
    inputs, labels = data
    #print(inputs)
    #print(labels)
    inputs=torch.autograd.Variable(inputs.to(device))  #数据和标签要放到GPU上
    labels=torch.autograd.Variable(labels.to(device))
                
            #optimizer.zero_grad()
                # forward + backward + optimize
    outputs = model_ft(inputs)
    #loss = criterion(outputs, labels)
                #print(loss)
    _,preds=torch.max(outputs.data,1)
    #print(preds)
    #print(labels.data)
    #print(preds==labels.data)
    for i in range(len(==.data)):
        if labels.data[i]==0:
            if preds[i]==0:
                c00+=1
            if preds[i]==1:
                c01+=1
            if preds[i]==2:
                c02+=1
        if labels.data[i]==1:
            if preds[i]==0:
                c10+=1
            if preds[i]==1:
                c11+=1
            if preds[i]==2:
                c12+=1
        if labels.data[i]==2:
            if preds[i]==0:
                c20+=1
            if preds[i]==1:
                c21+=1
            if preds[i]==2:
                c22+=1
    
    running_corrects+=(torch.sum(preds==labels.data)).item()
    
epoch_acc=running_corrects/len(dataset['valid'])
print(c00,c01,c02)
print(c10,c11,c12)
print(c20,c21,c22)
print(running_corrects)
print(len(dataset['valid']))
print(epoch_acc)                                                                              

881 0 13
5 900 16
9 11 2609
4390
4444
0.9878487848784878


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [3]:
import preresnet
model = preresnet.ResNet()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=model.to(device)

cfg [64, 64, 64, 256, 256, 64, 64, 256, 64, 64, 256, 128, 128, 512, 512, 128, 128, 512, 128, 128, 512, 128, 128, 512, 256, 256, 1024, 1024, 256, 256, 1024, 256, 256, 1024, 256, 256, 1024, 256, 256, 1024, 256, 256, 1024, 512, 512, 2048, 2048, 512, 512, 2048, 512, 512, 2048]


In [5]:
load_path='/home/are/data_cervical/pytorch_running/third/resnet50/Adam/resnet50_weight.pth'

model.load_state_dict(torch.load(load_path))
dataloader={'train':train_loader,'valid':valid_loader}
dataset={'train':train_dataset,'valid':valid_dataset}
running_corrects=0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
c00=0
c01=0
c02=0
c10=0
c11=0
c12=0
c20=0
c21=0
c22=0
for i, data in enumerate(dataloader['valid'], 0):
    model.eval()
    inputs, labels = data
    #print(inputs)
    #print(labels)
    inputs=torch.autograd.Variable(inputs.to(device))  #数据和标签要放到GPU上
    labels=torch.autograd.Variable(labels.to(device))
                
            #optimizer.zero_grad()
                # forward + backward + optimize
    outputs = model(inputs)
    #loss = criterion(outputs, labels)
                #print(loss)
    _,preds=torch.max(outputs.data,1)
    #print(preds)
    #print(labels.data)
    #print(preds==labels.data)
    for i in range(len(labels.data)):
        if labels.data[i]==0:
            if preds[i]==0:
                c00+=1
            if preds[i]==1:
                c01+=1
            if preds[i]==2:
                c02+=1
        if labels.data[i]==1:
            if preds[i]==0:
                c10+=1
            if preds[i]==1:
                c11+=1
            if preds[i]==2:
                c12+=1
        if labels.data[i]==2:
            if preds[i]==0:
                c20+=1
            if preds[i]==1:
                c21+=1
            if preds[i]==2:
                c22+=1
    
    running_corrects+=(torch.sum(preds==labels.data)).item()
    
epoch_acc=running_corrects/len(dataset['valid'])
print(c00,c01,c02)
print(c10,c11,c12)
print(c20,c21,c22)
print(running_corrects)
print(len(dataset['valid']))
print(epoch_acc)                                                                              

881 0 13
5 900 16
9 11 2609
4390
4444
0.9878487848784878


In [6]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [11]:
for k,m in enumerate(model.modules()):
    if isinstance(m,nn.Linear):
        print(m.bias.shape)

torch.Size([3])


In [12]:
print(model.layer1[1].bn1.weight.data)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0')
